<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem12_A_3D_frame_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

# =============================
# Problem data (from MATLAB)
# =============================
E  = 210e6
A  = 0.02
Iy = 10e-5
Iz = 20e-5
J  = 5e-5
G  = 84e6

nodeCoordinates = np.array([
    [0.0,  0.0,  0.0],
    [3.0,  0.0,  0.0],
    [0.0,  0.0, -3.0],
    [0.0, -4.0,  0.0]
], dtype=float)

elementNodes = np.array([
    [1, 2],
    [1, 3],
    [1, 4]
], dtype=int)

numberNodes = nodeCoordinates.shape[0]
numberElements = elementNodes.shape[0]
GDoF = 6 * numberNodes

U = np.zeros((GDoF, 1), dtype=float)
force = np.zeros((GDoF, 1), dtype=float)

# force vector (MATLAB: force(1)=-10; force(3)=20;)
force[0, 0] = -10.0
force[2, 0] =  20.0

# prescribed dof (MATLAB: 7:24)
prescribedDof = np.arange(6, GDoF)  # 0-based
allDof = np.arange(GDoF)
freeDof = np.setdiff1d(allDof, prescribedDof)

# =============================
# 3D frame stiffness assembly
# =============================
def element_dofs(n1, n2):
    """n1,n2 are 1-based node ids (like MATLAB). Return 0-based global dof indices (12 dof)."""
    i1 = n1 - 1
    i2 = n2 - 1
    dof1 = np.arange(6*i1, 6*i1 + 6)
    dof2 = np.arange(6*i2, 6*i2 + 6)
    return np.hstack([dof1, dof2])

def lambda_matrix(x1, y1, z1, x2, y2, z2):
    L = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)

    # special case: element parallel to global z (x1=x2 and y1=y2)
    if np.isclose(x1, x2) and np.isclose(y1, y2):
        if z2 > z1:
            Lambda = np.array([
                [0, 0,  1],
                [0, 1,  0],
                [-1,0,  0]
            ], dtype=float)
        else:
            Lambda = np.array([
                [0, 0, -1],
                [0, 1,  0],
                [1, 0,  0]
            ], dtype=float)
        return Lambda, L

    Cxx = (x2 - x1) / L
    CYx = (y2 - y1) / L
    CZx = (z2 - z1) / L

    D = np.sqrt(Cxx*Cxx + CYx*CYx)

    CXy = -CYx / D
    CYy =  Cxx / D
    CZy =  0.0

    CXz = -Cxx*CZx / D
    CYz = -CYx*CZx / D
    CZz =  D

    Lambda = np.array([
        [Cxx, CYx, CZx],
        [CXy, CYy, CZy],
        [CXz, CYz, CZz]
    ], dtype=float)

    return Lambda, L

def local_k_3dframe(E, A, Iz, Iy, G, J, L):
    # same scalars as MATLAB
    k1  = E*A/L
    k2  = 12*E*Iz/(L**3)
    k3  = 6*E*Iz/(L**2)
    k4  = 4*E*Iz/L
    k5  = 2*E*Iz/L
    k6  = 12*E*Iy/(L**3)
    k7  = 6*E*Iy/(L**2)
    k8  = 4*E*Iy/L
    k9  = 2*E*Iy/L
    k10 = G*J/L

    a = np.array([
        [k1, 0,  0],
        [0,  k2, 0],
        [0,  0,  k6]
    ], dtype=float)

    b = np.array([
        [0, 0,   0],
        [0, 0,  k3],
        [0, -k7, 0]
    ], dtype=float)

    c = np.array([
        [k10, 0,  0],
        [0,   k8, 0],
        [0,   0,  k4]
    ], dtype=float)

    d = np.array([
        [-k10, 0,  0],
        [0,    k9, 0],
        [0,    0,  k5]
    ], dtype=float)

    # k = [ a  b  -a   b
    #       b' c   b   d
    #      -a  b'  a  -b
    #       b  d' -b'  c ]
    k = np.block([
        [ a,      b,     -a,      b     ],
        [ b.T,    c,      b,      d     ],
        [ -a,     b.T,    a,     -b     ],
        [ b,      d.T,   -b.T,    c     ]
    ])
    return k

def formStiffness3Dframe(GDoF, numberElements, elementNodes, nodeCoordinates, E, A, Iz, Iy, G, J):
    K = np.zeros((GDoF, GDoF), dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]
        x1, y1, z1 = nodeCoordinates[n1-1, :]
        x2, y2, z2 = nodeCoordinates[n2-1, :]

        Lambda, L = lambda_matrix(x1, y1, z1, x2, y2, z2)
        k_local = local_k_3dframe(E, A, Iz, Iy, G, J, L)

        # R matrix (12x12)
        Z = np.zeros((3, 3), dtype=float)
        R = np.block([
            [Lambda, Z,     Z,     Z],
            [Z,      Lambda,Z,     Z],
            [Z,      Z,     Lambda,Z],
            [Z,      Z,     Z,     Lambda]
        ])

        k_global = R.T @ k_local @ R

        edof = element_dofs(n1, n2)
        K[np.ix_(edof, edof)] += k_global

    return K

stiffness = formStiffness3Dframe(GDoF, numberElements, elementNodes, nodeCoordinates, E, A, Iz, Iy, G, J)

# =============================
# Solve (same as MATLAB solution())
# =============================
Kff = stiffness[np.ix_(freeDof, freeDof)]
Ff  = force[freeDof, :]

Uf = np.linalg.solve(Kff, Ff)
U[freeDof, :] = Uf
U[prescribedDof, :] = 0.0

# =============================
# Print like MATLAB
# =============================
print("=== DISPLACEMENTS ===")
for i in range(GDoF):
    print(f"DOF {i+1:2d}: {U[i,0]: .6e}")


=== DISPLACEMENTS ===
DOF  1: -7.051478e-06
DOF  2: -6.653671e-08
DOF  3:  1.417696e-05
DOF  4:  1.447788e-06
DOF  5:  1.748584e-06
DOF  6:  1.136054e-06
DOF  7:  0.000000e+00
DOF  8:  0.000000e+00
DOF  9:  0.000000e+00
DOF 10:  0.000000e+00
DOF 11:  0.000000e+00
DOF 12:  0.000000e+00
DOF 13:  0.000000e+00
DOF 14:  0.000000e+00
DOF 15:  0.000000e+00
DOF 16:  0.000000e+00
DOF 17:  0.000000e+00
DOF 18:  0.000000e+00
DOF 19:  0.000000e+00
DOF 20:  0.000000e+00
DOF 21:  0.000000e+00
DOF 22:  0.000000e+00
DOF 23:  0.000000e+00
DOF 24:  0.000000e+00
